In [14]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()

In [13]:
connection.close()

In [11]:
update_inactive_D = '''update xxpo_vendor_master vm
set vendor_type = 'D'
from cspoms_facility_master fm
where vm.vendor_facility = fm.facility_code 
and fm.inactive_date IS not NULL
'''
post_cur.execute(update_inactive_D)
y = post_cur.rowcount
y

0

In [15]:
query = '''
WITH CrossJoinTemp AS (
--    select count(*)
--    from
--    (    
SELECT 
        LPAD(stg.bic_vendor_facility::VARCHAR, 3, '0') || LPAD(stg.bic_vendor_number::VARCHAR, 6, '0') AS home_destination_insert,
        af.facility_code::numeric AS facility_code,
        stg.*  
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric
--    		  ) as subquery

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, VendorOrderPoint AS (
    SELECT 
        vm.cs_vendor_id,
        temp.*
    FROM CrossJoinTemp temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

INSERT INTO XXPO_VENDOR_ORDER_POINT (
    CS_VENDOR_ID,
    MAIN_FACILITY,
    BUILD_TO_DAYS_SUPPLY_FLAG,
    DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS,

    LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7,

    APPLY_RATIONING,
    DEMAND_DAYS,
    LAG_DAYS,
    OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7,

    OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD,
    PULLBACK,

    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    CS_VENDOR_ID,
    MAIN_FACILITY,
    build_to_days_supply,
    DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS,

    LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7,

    APPLY_RATIONING,
    DEMAND_DAYS,
    LAG_DAYS,
    OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7,

    OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD,
    pullbks_available,

	-999, 
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM VendorOrderPoint
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
    BUILD_TO_DAYS_SUPPLY_FLAG = EXCLUDED.BUILD_TO_DAYS_SUPPLY_FLAG,
    DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN = EXCLUDED.USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS = EXCLUDED.USE_FAC_WHSE_SS,
    LEAD_TIME_OVRD_DAYS_1 = EXCLUDED.LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2 = EXCLUDED.LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3 = EXCLUDED.LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4 = EXCLUDED.LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5 = EXCLUDED.LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6 = EXCLUDED.LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7 = EXCLUDED.LEAD_TIME_OVRD_DAYS_7,
    APPLY_RATIONING = EXCLUDED.APPLY_RATIONING,
    DEMAND_DAYS = EXCLUDED.DEMAND_DAYS,
    LAG_DAYS = EXCLUDED.LAG_DAYS,
    OP_DAYS_1ST_RUN_1 = EXCLUDED.OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2 = EXCLUDED.OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3 = EXCLUDED.OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4 = EXCLUDED.OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5 = EXCLUDED.OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6 = EXCLUDED.OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7 = EXCLUDED.OP_DAYS_1ST_RUN_7,
    OP_DAYS_2ND_RUN_1 = EXCLUDED.OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2 = EXCLUDED.OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3 = EXCLUDED.OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4 = EXCLUDED.OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5 = EXCLUDED.OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6 = EXCLUDED.OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7 = EXCLUDED.OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR = EXCLUDED.SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD = EXCLUDED.HAWAII_TRANS_METHOD,
    PULLBACK = EXCLUDED.PULLBACK,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    CREATION_DATE = EXCLUDED.CREATION_DATE,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    CREATED_BY = EXCLUDED.CREATED_BY,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY;
'''
post_cur.execute(query)
x = post_cur.rowcount
x

799204

In [ ]:
print(x)